In [15]:
import time
import matplotlib.pyplot as plt
import pandas as pd

import networkx as nx
import graph_tool.all as gt

import cc_graph_ops

In [16]:
INPUT_FILE = 'fdg_input_file.json'

In [17]:
g = cc_graph_ops.create_graph_from_file(INPUT_FILE, format='gt')

In [18]:
# Overflow for 32-bit ints
weight_long = g.ep['weight'].copy(value_type='long')

In [19]:
coef, stdev = gt.global_clustering(g, weight=weight_long)

In [20]:
coefs, stdevs = dict(), dict()

coefs['all'] = coef
stdevs['all'] = stdev

In [21]:
quota = 10
proportion = 0.3
min_subgraph_size = 100

In [22]:
licenses = list(cc_graph_ops.get_licenses(g))
subgraphs = cc_graph_ops.all_license_subgraphs(g, licenses, quota=quota, proportion=proportion)

In [23]:
for license, subg in subgraphs.items():
    if subg.num_vertices() < min_subgraph_size:
        continue
    coef, stdev = gt.global_clustering(subg, weight=weight_long)
    coefs[license] = coef
    stdevs[license] = stdev

In [24]:
size = {license: subg.num_vertices() for license, subg in subgraphs.items()}
size['all'] = g.num_vertices()

In [25]:
keys = list(coefs.keys())
keys.sort(key=lambda x: -coefs[x])

In [26]:
table = [[k, size[k], coefs[k]] for k in keys]
global_clustering_df = pd.DataFrame(table, columns=['License', 'Size', 'Clustering'])
global_clustering_df = global_clustering_df.set_index('License')
display(global_clustering_df)

,Size,Clustering
License,,
"('by-nc-sa', '2.5')",1118,0.793559
"('by-sa', '3.0')",6514,0.601791
all,235863,0.440094
"('cc0', '1.0')",893,0.410368
"('by-nc-nd', '4.0')",7387,0.404350
"('by', '3.0')",5722,0.403254
"('by-nd', '4.0')",629,0.328749
"('by-sa', '4.0')",7066,0.323638
"('by', '4.0')",9085,0.278524


In [27]:
local_coefs = dict()
for license in coefs:
    if license != 'all':
        local_clusters = gt.local_clustering(subgraphs[license], weight=weight_long)
        local_clusters = {v: local_clusters[v] for v in subgraphs[license].vertices()}
        local_clusters = sorted(list(local_clusters.items()), key=lambda x: -x[1])
        local_coefs[license] = local_clusters

In [28]:
for license in global_clustering_df.index:
    if license != 'all':
        print(license, global_clustering_df.loc[license, 'Clustering'])
        data = []
        for v, coef in local_coefs[license][:10]:
            data.append([subgraphs[license].vp['provider_domain'][v], coef])
        display(pd.DataFrame(data, columns=['Domain', 'Clustering']))
        print()

('by-nc-sa', '2.5') 0.7935592444906805


,Domain,Clustering
0,shun.im,218.000000
1,www.qiusi.me,72.666667
2,www.surveillanceincanada.org,36.000000
3,pochistvane.com,21.619565
4,www.xj123.info,16.769231
5,www.danielgarciaperis.cat,15.912334
6,praxis.tw,12.285714
7,xn--czq75pvv1aj5c.org,9.750000
8,felixc.at,9.333333
9,blog.imprenditore.me,6.914286



('by-sa', '3.0') 0.6017905751196228


,Domain,Clustering
0,contentedness.net,2.081500e+07
1,50days.org,2.081500e+07
2,www.linke-piraten-hd.de,2.081500e+07
3,hejlt.org,1.040750e+07
4,enskedebilder.com,1.040750e+07
5,www.infoturia.com,6.938334e+06
6,www.daemmen-und-sanieren.de,6.938334e+06
7,fortunengineering.com,6.938334e+06
8,www.adhugger.net,6.938334e+06
9,geschichtskasten.kastenwesen.com,5.203764e+06



('cc0', '1.0') 0.4103683368256483


...

,Domain,Clustering
0,www.pcihispano.com,15005.892857
1,volatilesystems.org,8753.125000
2,textilegeschichten.net,5016.109091
3,botons.eu,3654.000000
4,www.lapsusmentis.com,3153.666667
5,ojs.correspondenciasyanalisis.com,1759.294872
6,etherboot.org,1480.283333
7,stinpriza.org,1227.259259
8,www.pedagomosaique.com,1042.357143
9,www.teawiki.net,915.418776



('pdm', '1.0') 0.001155464633078091


,Domain,Clustering
0,www.europeana.eu,6.426225
1,www.jugendhilfeportal.de,2.385621
2,dija.faktor-e.domainfactory-kunde.de,0.166358
3,biblioteca.galiciana.gal,0.098662
4,www.edu.xunta.gal,0.092572
5,m.dija.de,0.081716
6,fdanj.nlm.nih.gov,0.039724
7,cartotecadigital.icgc.cat,0.000739
8,cartotecadigital.icc.cat,0.000554
9,openimages.eu,0.000438



('by-nd', '2.0') 0.0


,Domain,Clustering
0,freerepublic.com,0.0
1,www.teemingbrain.com,0.0
2,clubdumillenaire.fr,0.0
3,www.zakon-pritazlivosti.cz,0.0
4,uncontrollinglove.com,0.0
5,davidcantone.com,0.0
6,www.livewritethrive.com,0.0
7,www.reha-trans.fr,0.0
8,semeunacte.com,0.0
9,www.empire-of-books.com,0.0
